In [1]:
import os
import random
import warnings
from collections import defaultdict
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from Classes import *
from graphviz import Digraph
from pandas.errors import SettingWithCopyWarning
from RunningHorizon import *
from aaai_experiments import *
from sklearn.model_selection import train_test_split
from utils import *
from random import sample

pd.set_option('display.max_rows', None)

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(
    "ignore",
    message="Could not infer format, so each element will be parsed individually, falling back to dateutil. To ensure parsing is consistent and as-expected, please specify a format.",
)


In [24]:
import time
from threading import Thread
from typing import Dict, List, Optional, Any
import pandas as pd
import numpy as np

def run_algorithm_with_timeout(sync_prod, algorithm: str, timeout: Optional[int] = None) -> Optional[Dict[str, Any]]:
    """
    Runs a specified search algorithm on the provided sync product with a timeout.
    Handles exceptions and returns None if the algorithm fails or times out.
    """
    result = {}
    
    def target():
        try:
            if algorithm == 'astar':
                result['alignment'], result['cost'], result['nodes_popped'] = sync_prod.astar_search()
            elif algorithm == 'astar_extended':
                result['alignment'], result['cost'], result['nodes_popped'] = sync_prod.astar_incremental()
            elif algorithm == 'reach':
                result['alignment'], result['cost'], result['nodes_popped'] = sync_prod.reach_search()
        except Exception as e:
            result['error'] = e
    
    thread = Thread(target=target)
    thread.start()
    thread.join(timeout=timeout)
    
    if thread.is_alive():
        print(f"Timeout occurred for {algorithm}.")
        return None
    
    if 'error' in result:
        print(f"Error occurred for {algorithm}: {result['error']}")
        return None
    
    return {'cost': result.get('cost'), 'nodes_popped': result.get('nodes_popped')}

def compare_search_algorithms(
    df: pd.DataFrame,
    df_name: str,
    cost_function: Optional[Any] = None,  # Replace 'Any' with the actual type if known
    n_train_traces: int = 10,
    n_test_traces: int = 10,
    train_cases: Optional[List[str]] = None,
    test_cases: Optional[List[str]] = None,
    random_seed: int = 304,
    non_sync_penalty: int = 1,
    only_return_model: bool = False,
    allow_intersection: bool = False,
    read_model_from_file: bool = False,
    model_path: Optional[str] = None,
    activity_mapping_dict: Optional[Dict[str, str]] = None,
    algorithms: Optional[List[str]] = None,
    time_limit: Optional[int] = None
) -> Optional[Any]:  # Replace 'Any' with the actual return type of prepare_model if known
    """
    Function to compare specified search algorithms (A*, A* Extended, Reach) 
    by either training a model on the training data or reading it from a file, 
    and then evaluating it on the test data.
    """
    if algorithms is None:
        algorithms = ['astar', 'astar_extended', 'reach']

    cost_function = select_cost_function(cost_function)
    np.random.seed(random_seed)
    
    result = train_test_log_split(
        df, 
        n_train_traces=n_train_traces, 
        n_test_traces=n_test_traces,
        train_traces=train_cases,
        test_traces=test_cases,
        random_selection=(train_cases is None and test_cases is None), 
        random_seed=random_seed,
        allow_intersection=allow_intersection
    )
    
    train_df, test_df = result['train_df'], result['test_df']
    
    if read_model_from_file:
        if not model_path or not activity_mapping_dict:
            raise ValueError("Both 'model_path' and 'activity_mapping_dict' must be provided when reading a model from a file.")
        
        model = generate_model_from_file(
            model_path,
            activity_mapping_dict=activity_mapping_dict,
            return_markings=False
        )
    else:
        model = prepare_model(train_df, non_sync_penalty)
    
    if only_return_model:
        return model

    model = add_transition_mappings_to_model(model)
 
    # Group similar traces
    trace_dict, lookup_dict = group_similar_traces(test_df)

    # Initialize results dictionary
    results = {alg: {'cost': [], 'time': [], 'nodes_popped': [], 'case_id': []} for alg in algorithms}
    
    computed_results = {}
    
    print(f"Overall computing statistics for {len(trace_dict)} traces")
    for trace, cases in trace_dict.items():
        representative_case = cases[0]
        true_trace_df = test_df[test_df['case:concept:name'] == representative_case].reset_index(drop=True)
        true_trace_df['probs'] = [[1.0] for _ in range(len(true_trace_df))]
        stochastic_trace = construct_stochastic_trace_model(true_trace_df, non_sync_penalty)
        sync_prod = SyncProduct(model, stochastic_trace, cost_function=cost_function)
        
        computed_results[trace] = {}
        
        for alg in algorithms:
            computed_results[trace][alg] = run_algorithm_with_timeout(
                sync_prod, alg, timeout=time_limit
            )
        
        # Store results for all cases with the same trace
        for case_id in cases:
            for alg in algorithms:
                if computed_results[trace][alg]:
                    results[alg]['cost'].append(computed_results[trace][alg]['cost'])
                    results[alg]['time'].append(time_limit)  # Store the time limit as the time taken
                    results[alg]['nodes_popped'].append(computed_results[trace][alg]['nodes_popped'])
                else:
                    results[alg]['cost'].append(None)
                    results[alg]['time'].append(None)
                    results[alg]['nodes_popped'].append(None)
                results[alg]['case_id'].append(case_id)

    # Save results as DataFrames with the dataframe name included in the filename
    for alg in algorithms:
        save_results_as_dataframe(results[alg], df_name, alg)

    return None  # Return None if only_return_model is False and no other return is specified


In [2]:
PATH = 'C:\\Users\\User\\Jupyter Projects\\Research_2\\Datasets'
os.chdir(PATH)

df_name = 'bpi_2012.csv'
df = pd.read_csv(df_name)
df = map_to_string_numbers(df, map_strings_to_integer_strings=True, return_map_dict=False)
df['case:concept:name'] = df.groupby('case:concept:name').ngroup().astype(str)
df_filtered = filter_log_efficiently(df, min_len=50, max_len=80, n_traces=15, random_seed=304)

In [3]:
compare_search_algorithms(df_filtered, df_name=df_name, n_train_traces=5, n_test_traces=5,
                          algorithms=['astar_extended'], time_limit=10)

The train cases are: ['237186', '86447', '175911', '143534', '187000'] 

The test cases are: ['144668', '11578', '392', '231334', '2077'] 

Starting to compute mandatory transitions...
Mandatory transitions computed in 0.0133 seconds
Starting to compute alive transitions...
Alive transitions computed in 0.0011 seconds
Overall computing statistics for 5 traces
Timeout occurred for astar_extended.
Timeout occurred for astar_extended.
Timeout occurred for astar_extended.


## Video Datasets

In [2]:
df, softmax_lst = prepare_df('gtea')
trace_groups = group_similar_traces(df)
print(trace_groups)
df.head()

          case_list  trace_length
0    [8, 9, 10, 11]          2009
1      [4, 5, 6, 7]           943
2  [12, 13, 14, 15]          1178
3  [24, 25, 26, 27]           718
4  [20, 21, 22, 23]          1235
5  [16, 17, 18, 19]          1384
6      [0, 1, 2, 3]          1643


,case:concept:name,concept:name
0,0,10
1,0,10
2,0,10
3,0,10
4,0,10


In [3]:
sampled_cases = sample_cases(trace_groups, n=1)
print(sampled_cases)

filtered_df = filter_df_by_cases(df, sampled_cases)
filtered_softmax_lst = filter_softmax_matrices(softmax_lst, sampled_cases)

hist_prob_dict = build_conditioned_prob_dict(filtered_df, max_hist_len=3)

filtered_df.head()

['2', '7', '10', '13', '18', '23', '26']


,case:concept:name,concept:name
3286,2,10
3287,2,10
3288,2,10
3289,2,10
3290,2,10


In [4]:
dijkstra_distances, reach_distances = compare_stochastic_vs_argmax_random_indices(
    df,
    softmax_lst=None,
    n_indices=5,
    activity_prob_threshold=0.02,
    cost_function=None,
    random_seed=304,
    test_cases=None,
    train_cases=None,
    n_test_traces=15,
    n_train_traces=5,
    allow_intersection=False,
    include_duplicate_traces=True
)

The train cases are: ['11', '0', '5', '17', '12'] 

The test cases are: ['20', '3', '8', '2', '26', '25', '15', '10', '23', '27', '24', '22', '19', '18', '21'] 



,case:concept:name,concept:name,probs
0,20,7,[1.0]
1,20,10,[1.0]
2,20,5,[1.0]
3,20,10,[1.0]
4,20,6,[1.0]


,case:concept:name,concept:name,probs
0,3,3,[1.0]
1,3,10,[1.0]
2,3,9,[1.0]
3,3,10,[1.0]
4,3,2,[1.0]


,case:concept:name,concept:name,probs
0,8,1,[1.0]
1,8,10,[1.0]
2,8,4,[1.0]
3,8,6,[1.0]
4,8,3,[1.0]


,case:concept:name,concept:name,probs
0,2,0,[1.0]
1,2,10,[1.0]
2,2,10,[1.0]
3,2,2,[1.0]
4,2,9,[1.0]


...

,case:concept:name,concept:name,probs
0,24,7,[1.0]
1,24,0,[1.0]
2,24,2,[1.0]
3,24,10,[1.0]
4,24,0,[1.0]


,case:concept:name,concept:name,probs
0,22,5,[1.0]
1,22,0,[1.0]
2,22,6,[1.0]
3,22,3,[1.0]
4,22,2,[1.0]


,case:concept:name,concept:name,probs
0,19,1,[1.0]
1,19,7,[1.0]
2,19,10,[1.0]
3,19,10,[1.0]
4,19,10,[1.0]


,case:concept:name,concept:name,probs
0,18,2,[1.0]
1,18,9,[1.0]
2,18,0,[1.0]
3,18,9,[1.0]
4,18,9,[1.0]


,case:concept:name,concept:name,probs
0,21,3,[1.0]
1,21,2,[1.0]
2,21,3,[1.0]
3,21,10,[1.0]
4,21,6,[1.0]


In [5]:
dijkstra_distances

[3.00000207,
 2.000003079999999,
 2.0000030499999992,
 1.0000040899999996,
 1.0000040899999996,
 3.00000207,
 3.00000207,
 4.00000106,
 3.00000207,
 1.0000040899999991,
 2.00000308,
 5.00000107,
 5.099999999999999e-06,
 2.000004099999999,
 4.00000106]

In [6]:
reach_distances

[3.00000207,
 2.000003079999999,
 2.0000030499999992,
 1.0000040899999998,
 1.0000040899999996,
 3.00000207,
 3.00000207,
 4.00000106,
 3.00000207,
 1.0000040899999991,
 2.00000308,
 5.00000107,
 5.099999999999999e-06,
 2.000004099999999,
 4.00000106]

In [5]:
model = compare_stochastic_vs_argmax_random_indices(
    df,
    softmax_lst=None,
    n_indices=5,
    activity_prob_threshold=0.02,
    cost_function=None,
    random_seed=202,
    test_cases=None,
    train_cases=None,
    n_test_traces=1,
    n_train_traces=5,
    allow_intersection=False,
    include_duplicate_traces=True,
    only_return_model=True
)

The train cases are: ['4', '27', '0', '25', '2'] 

The test cases are: ['9'] 



In [10]:
# only the part of the marking corresponding to the model i.e., marking[:len(model.places)]
marking_tuple = (0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,) 
visualize_petri_net(model, marking=marking_tuple, output_path="./model_with_tokens")

Visualization saved to: ./model_with_tokens


In [7]:
model.places

[source,
 p_5,
 p_6,
 p_7,
 p_8,
 p_10,
 p_12,
 p_13,
 p_14,
 p_15,
 p_16,
 p_18,
 p_19,
 p_20,
 p_21,
 p_22,
 p_23,
 p_24,
 p_25,
 p_26,
 p_28,
 p_29,
 p_30,
 p_31,
 sink]

## compare windows refactored

In [1]:

from aaai_experiments import *

In [2]:
PATH = 'C:\\Users\\User\\Jupyter Projects\\Research_2\\Datasets'
os.chdir(PATH)

df_name = 'bpi_2012.csv'

In [3]:
def compare_window_based_baselines(
    df_name: str = '',
    model_path: str = '',    
    n_train_traces: int = None,
    n_test_traces: int = None,
    train_cases: List[str] = None,
    test_cases: List[str] = None,
    window_lengths_lst: List[int] = None,
    n_final_markings_lst: List[int] = None,
    only_return_model: bool = False,
    window_overlap: int = 0,
    read_model_from_file: bool = False,
    non_sync_penalty: int = 1,
    allow_intersection: bool = True,
    cost_function: Any = None,
    use_heuristics=False,
    max_len=None,
    min_len=None,
    n_traces=None,
    random_seed=304,
    map_dict=None,
    explor_reward=0.0000001
) -> Union[pd.DataFrame, Tuple]:

    df, map_dict = load_and_preprocess_log(df_name, min_len=min_len, max_len=max_len, n_traces=n_traces, random_seed=random_seed)

    cost_function = select_cost_function(cost_function)
    np.random.seed(random_seed)
    
    result = train_test_log_split(
        df, 
        n_train_traces=n_train_traces, 
        n_test_traces=n_test_traces,
        train_traces=train_cases,
        test_traces=test_cases,
        random_selection=(train_cases is None and test_cases is None), 
        random_seed=random_seed,
        allow_intersection=allow_intersection
    )
    
    train_df, test_df = result['train_df'], result['test_df']
    test_df['probs'] = [[1.0] for _ in range(len(test_df))]
    
    if read_model_from_file:
        if not model_path or not map_dict:
            raise ValueError("Both 'model_path' and 'map_dict' must be provided when reading a model from a file.")
        
        model = generate_model_from_file(
            model_path,
            activity_mapping_dict=map_dict,
            return_markings=False
        )
    else:
        model = prepare_model(train_df, non_sync_penalty)

    if use_heuristics:
        model = add_transition_mappings_to_model(model)
    
    results = wcb_perform_conformance_checking(
        test_df, model, window_lengths_lst, n_final_markings_lst,
        explor_reward, window_overlap, use_heuristics,
        cost_function
    )

    res_df = wcb_convert_to_dataframe(results)
    return res_df
    
# def wcb_initialize_result_dictionaries() -> Dict[str, defaultdict]:
#     return {
#         'cost': defaultdict(list),
#         'time': defaultdict(list),
#         'nodes_opened': defaultdict(list),
#         'stochastic_acc': defaultdict(list),
#         'argmax_acc': defaultdict(list),
#         'alignment': defaultdict(list),
#         'alignment_weights': defaultdict(list)
#     }
                                     
def wcb_perform_conformance_checking(
    test_df: pd.DataFrame, 
    net: Any,  # Assuming PetriNet is some class defined elsewhere
    window_lengths_lst: List[int],
    n_final_markings_lst: List[int],
    explor_reward: float, 
    window_overlap: int,
    use_heuristics: bool, 
    cost_function: Any
) -> Dict[str, defaultdict]:
    
    results = defaultdict(lambda: defaultdict(list))
    
    trace_dict, lookup_dict = group_similar_traces(test_df)
    total_traces = len(trace_dict)

    for window_len in window_lengths_lst:
        for n_markings in n_final_markings_lst:
            print(f'Evaluating variant: n_markings={n_markings}, window_len={window_len}')
            
            for idx, (trace, cases) in enumerate(trace_dict.items(), 1):
                print(f'\rComputing trace {idx}/{total_traces}', end='')

                representative_case = cases[0]
                trace_df = test_df[test_df['case:concept:name'] == representative_case]
                
                start_time = time.time()
                dist, full_alignment, nodes_opened = horizon_based_conformance(
                    net, trace_df, window_len=window_len, n_unique_final_markings=n_markings,
                    explor_reward=explor_reward, window_overlap=window_overlap,
                    cost_function=cost_function, use_heuristics=use_heuristics
                )
                computation_time = time.time() - start_time
                
                for case in cases:
                    wcb_update_results(case, results, n_markings, window_len, dist, nodes_opened, computation_time, full_alignment)
            
            if window_len is None:
                break

        print()

    return results


def wcb_update_results(case, results, n_markings, window_len, dist, nodes_opened, computation_time, full_alignment):
    key = f'window_{window_len}_markings_{n_markings}'
    results[key]['case_id'].append(case)
    results[key]['cost'].append(dist)
    results[key]['time'].append(computation_time)
    results[key]['nodes_opened'].append(nodes_opened)
    results[key]['alignment'].append(full_alignment)


def wcb_convert_to_dataframe(data):
    """
    Converts a nested dictionary with multiple outer keys into a pandas DataFrame.
    Rounds the 'cost' and 'time' columns to 5 decimal places and prefixes these columns with the outer key.
    Retains only 'case_id', 'cost', 'time', and 'nodes_opened' columns.
    
    Args:
        data (dict): A dictionary where the keys are outer keys and the values are dictionaries containing lists.
    
    Returns:
        pd.DataFrame: A DataFrame with prefixed columns for each outer key.
    """
    all_dfs = []
    for key, inner_dict in data.items():
        # Retain only the relevant columns
        filtered_dict = {
            'case_id': inner_dict.get('case_id'),
            'cost': inner_dict.get('cost'),
            'time': inner_dict.get('time'),
            'nodes_opened': inner_dict.get('nodes_opened')
        }
        
        # Convert the filtered dictionary to a DataFrame
        df = pd.DataFrame(filtered_dict)
        
        # Round the 'cost' and 'time' columns to 5 decimal places, if they exist
        if 'cost' in df.columns:
            df['cost'] = df['cost'].round(5)
        if 'time' in df.columns:
            df['time'] = df['time'].round(5)
        
        # Prefix columns with the outer key, except for 'case_id'
        df = df.rename(columns={
            'cost': f'{key}_cost',
            'time': f'{key}_time',
            'nodes_opened': f'{key}_nodes_opened'
        })
        
        # Append the DataFrame to the list
        all_dfs.append(df)
    
    # Merge all DataFrames on 'case_id'
    final_df = pd.concat(all_dfs, axis=1)
    
    # Remove duplicate 'case_id' columns that might appear due to concatenation
    final_df = final_df.loc[:,~final_df.columns.duplicated()]
    
    return final_df

In [3]:
res = compare_window_based_baselines('bpi_2012.csv', n_train_traces=10, n_test_traces=2, window_lengths_lst=[30], n_final_markings_lst=[1], max_len=120, min_len=80, use_heuristics=False)

The train cases are: ['19181', '244810', '11346', '183434', '187011'] ... [and 5 more]

The test cases are: ['19181', '244810'] 

Evaluating variant: n_markings=1, window_len=30
Computing trace 2/2


In [5]:
res

,case_id,window_30_markings_1_cost,window_30_markings_1_time,window_30_markings_1_nodes_opened
0,19181,0.00009,6.42961,58617
1,244810,0.00009,5.10678,45895


In [10]:
res

,case_id,window_10_markings_1_cost,window_10_markings_1_time,window_10_markings_1_nodes_opened
0,10415,0.00011,1.30153,10657
1,11346,0.00011,1.14992,10307
2,11475,0.00011,4.82698,36560
3,13261,0.00011,1.22224,11823
4,13297,0.00009,5.65527,45519
5,183434,0.00012,1.48897,14437
6,187011,0.00010,1.36129,12680
7,191137,0.00011,3.66703,31340
8,19181,0.00009,6.88472,54668
9,19183,0.00008,4.82683,38411


## Working refactored code

In [8]:
def compare_window_based_baselines(
    df: pd.DataFrame,
    sk_df: pd.DataFrame = None,
    sftmax_lst: List[np.ndarray] = None,
    n_train_traces: int = None,
    n_test_traces: int = None,
    random_trace_selection: bool = False,
    train_traces: List[str] = None,
    test_traces: List[str] = None,
    window_lengths_lst: List[int] = None,
    n_final_markings_lst: List[int] = None,
    seed_1: int = 101,
    seed_2: int = 202,
    as_df: bool = True,
    viz_proc_tree: bool = False,
    return_alignment_res_dict: bool = False,
    return_model: bool = False,
    explor_reward: float = 0.000,
    window_overlap: int = 0,
    read_model_from_file: bool = False,
    file_path: str = '',
    activity_mapping_dict: Dict[str, str] = None,
    use_trace_frequencies: bool = False,
    return_agg_res_dict: bool = False,
    reachability_heuristic: bool = False,
    trace_recovery: bool = False,
    map_activities_to_strings: bool = True,
    include_duplicate_traces: bool = True,
    non_sync_penalty: int = 1,
    allow_train_test_intersection: bool = True,
    cost_function: Any = None,
    n_sampling_indices: int = 5,
    round_precision: int = 3,
    activity_prob_threshold: float = 0,
    sequential_sampling: bool = False,
    return_test_df: bool = False,
    return_test_ground_truth: bool = True,
    return_alignment_weights_dict: bool = True,
    logarithmic_scaling_constant: float = 4.6,
    hist_prob_dict: Dict[Any, Any] = None
) -> Union[pd.DataFrame, Tuple]:
    
    result_dicts = wcb_initialize_result_dictionaries()
    window_lengths_lst, n_final_markings_lst = wcb_set_default_values(window_lengths_lst, n_final_markings_lst)
    cost_function = wcb_prepare_cost_function(cost_function, logarithmic_scaling_constant)
    
    train_df, test_df, test_ground_truth, test_sftmax_lst = wcb_split_log(
        df, sk_df, sftmax_lst, n_train_traces, n_test_traces, train_traces, test_traces,
        random_trace_selection, seed_1, include_duplicate_traces, allow_train_test_intersection
    )
    
    test_df = wcb_process_test_data(
        test_df, test_ground_truth, test_sftmax_lst, n_sampling_indices,
        sequential_sampling, seed_1, round_precision, activity_prob_threshold
    )
    
    net, init_marking, final_marking = wcb_prepare_process_model(
        train_df, read_model_from_file, file_path, activity_mapping_dict,
        non_sync_penalty, cost_function
    )
    
    net_details_dict = wcb_prepare_reachability_heuristic(net, reachability_heuristic)
    
    results = wcb_perform_conformance_checking(
        test_df, net, window_lengths_lst, n_final_markings_lst,
        explor_reward, window_overlap, net_details_dict, reachability_heuristic,
        cost_function, hist_prob_dict, test_ground_truth
    )
    
    final_df = wcb_process_results(results, result_dicts, as_df, use_trace_frequencies, df, test_df)
    
    return handle_return_logic(
        final_df,
        alignment_res_dict=results['alignment'],
        model=(net, init_marking, final_marking),
        return_alignment_res_dict=return_alignment_res_dict,
        return_model=return_model,
        return_test_df=return_test_df,
        test_df=test_df,
        return_test_ground_truth=return_test_ground_truth,
        test_ground_truth=test_ground_truth,
        alignment_weights_dict=results['alignment_weights'],
        return_alignment_weights_dict=return_alignment_weights_dict
    )

def wcb_initialize_result_dictionaries() -> Dict[str, defaultdict]:
    return {
        'cost': defaultdict(list),
        'time': defaultdict(list),
        'nodes_opened': defaultdict(list),
        'stochastic_acc': defaultdict(list),
        'argmax_acc': defaultdict(list),
        'alignment': defaultdict(list),
        'alignment_weights': defaultdict(list)
    }

def wcb_set_default_values(window_lengths_lst: List[int], n_final_markings_lst: List[int]) -> Tuple[List[int], List[int]]:
    return (
        [3] if window_lengths_lst is None else window_lengths_lst,
        [1] if n_final_markings_lst is None else n_final_markings_lst
    )

def wcb_prepare_cost_function(cost_function: Any, logarithmic_scaling_constant: float) -> Any:
    if cost_function == 'logarithmic':
        return lambda x: -np.log(x) / logarithmic_scaling_constant
    return cost_function

def wcb_split_log(df: pd.DataFrame, sk_df: pd.DataFrame, sftmax_lst: List[np.ndarray],
                  n_train_traces: int, n_test_traces: int, train_traces: List[str], test_traces: List[str],
                  random_trace_selection: bool, seed_1: int, include_duplicate_traces: bool,
                  allow_train_test_intersection: bool) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, List[np.ndarray]]:
    result = train_test_log_split(
        df, n_train_traces, n_test_traces, train_traces, test_traces,
        random_trace_selection, seed_1, include_duplicate_traces,
        allow_train_test_intersection, sk_df, sftmax_lst
    )
    
    train_df = result['train_df']
    test_df = result['test_df']
    test_ground_truth = result.get('test_ground_truth', None)
    test_sftmax_lst = result.get('test_sftmax_lst', None)
    
    if test_ground_truth is None:
        test_ground_truth = test_df.copy()
    
    if test_sftmax_lst is None:
        test_sftmax_lst = [None] * len(test_df['case:concept:name'].unique())
    
    return train_df, test_df, test_ground_truth, test_sftmax_lst

def wcb_process_test_data(test_df: pd.DataFrame, test_ground_truth: pd.DataFrame, test_sftmax_lst: List[np.ndarray],
                          n_sampling_indices: int, sequential_sampling: bool, seed_1: int,
                          round_precision: int, activity_prob_threshold: float) -> pd.DataFrame:
    if test_sftmax_lst[0] is not None:
        test_sftmax_lst = convert_tensors_to_numpy(test_sftmax_lst)
        test_ground_truth, test_sftmax_lst = select_random_indices_in_log_and_sftm_matrices_lst(
            test_df, test_sftmax_lst, n_sampling_indices, 
            sequential_sampling=sequential_sampling, random_seed=seed_1
        )
        test_df = wcb_process_stochastic_traces(test_ground_truth, test_sftmax_lst, round_precision, activity_prob_threshold)
    
    if 'probs' not in test_df.columns:
        test_df['probs'] = [[1.0] for _ in range(len(test_df))]
    
    return test_df

def wcb_process_stochastic_traces(test_ground_truth: pd.DataFrame, test_sftmax_lst: List[np.ndarray],
                                  round_precision: int, activity_prob_threshold: float) -> pd.DataFrame:
    stochastic_trace_df_lst = []
    test_traces_cases = list(test_ground_truth['case:concept:name'].unique())
    for idx, case in enumerate(test_traces_cases):
        stochastic_trace_df = sfmx_mat_to_sk_trace(test_sftmax_lst[idx], case,
                                                   round_precision=round_precision,
                                                   threshold=activity_prob_threshold)
        stochastic_trace_df_lst.append(stochastic_trace_df)
    return pd.concat(stochastic_trace_df_lst)

def wcb_prepare_process_model(train_df: pd.DataFrame, read_model_from_file: bool, file_path: str,
                              activity_mapping_dict: Dict[str, str], non_sync_penalty: int,
                              cost_function: Any) -> Tuple[PetriNet, Marking, Marking]:
    if read_model_from_file:
        if activity_mapping_dict is None:
            raise ValueError('To generate a process model from a file you must pass a mapping dictionary to be consistent with the log')
        net, init_marking, final_marking = generate_model_from_file(file_path, activity_mapping_dict=activity_mapping_dict)
    else:
        net, init_marking, final_marking, _ = prepare_process_model(train_df, non_sync_penalty=non_sync_penalty, cost_function=cost_function)
    
    return net, init_marking, final_marking

def wcb_prepare_reachability_heuristic(net: PetriNet, reachability_heuristic: bool) -> Dict[Any, Any]:
    if reachability_heuristic:
        print('Started computing net_details_dict')
        net_details_dict = map_markings_to_reachable_transitions(net)
        print('Finished computing net_details_dict')
    else:
        net_details_dict = None
    return net_details_dict

def wcb_perform_conformance_checking(test_df: pd.DataFrame, net: PetriNet, window_lengths_lst: List[int],
                                     n_final_markings_lst: List[int], explor_reward: float, window_overlap: int,
                                     net_details_dict: Dict[Any, Any], reachability_heuristic: bool,
                                     cost_function: Any, hist_prob_dict: Dict[Any, Any],
                                     test_ground_truth: pd.DataFrame) -> Dict[str, defaultdict]:
    results = wcb_initialize_result_dictionaries()
    
    for window_len in window_lengths_lst:
        for n_markings in n_final_markings_lst:
            print(f'Evaluating variant: n_markings={n_markings}, window_len={window_len}')
            
            for case in test_df['case:concept:name'].unique():
                trace_df = test_df[test_df['case:concept:name'] == case]
                
                start_time = time.time()
                dist, full_alignment, nodes_opened = horizon_based_conformance(
                    net, trace_df, window_len=window_len, n_unique_final_markings=n_markings,
                    explor_reward=explor_reward, window_overlap=window_overlap,
                    net_details_dict=net_details_dict, reachability_heuristic=reachability_heuristic, 
                    cost_function=cost_function, hist_prob_dict=hist_prob_dict
                )
                computation_time = time.time() - start_time
                
                wcb_update_results(results, n_markings, window_len, dist, nodes_opened, computation_time, full_alignment)
                
                if test_ground_truth is not None:
                    wcb_update_accuracy_results(results, n_markings, window_len, full_alignment, trace_df, test_ground_truth, case)
            
            if window_len is None:
                break
    
    return results

def wcb_update_results(results: Dict[str, defaultdict], n_markings: int, window_len: int,
                       dist: float, nodes_opened: int, computation_time: float, full_alignment: List[Any]):
    key = (n_markings, window_len)
    results['cost'][key].append(dist)
    results['time'][key].append(computation_time)
    results['nodes_opened'][key].append(nodes_opened)
    results['alignment'][key].append([t.name if isinstance(t, Transition) else t for t in full_alignment] + [dist])
    results['alignment_weights'][key].append([t.weight if isinstance(t, Transition) else t for t in full_alignment] + [dist])


def wcb_update_accuracy_results(results: Dict[str, defaultdict], n_markings: int, window_len: int,
                                full_alignment: List[Any], trace_df: pd.DataFrame,
                                test_ground_truth: pd.DataFrame, case: str):
    key = (n_markings, window_len)
    clean_alignment = [t for t in full_alignment if isinstance(t, Transition)]
    true_trace_df = test_ground_truth[test_ground_truth['case:concept:name'] == case]
    argmax_accuracy = compute_argmax_accuracy(trace_df, true_trace_df)
    stochastic_accuracy = compute_alignment_accuracy(clean_alignment, true_trace_df)
    results['argmax_acc'][key].append(argmax_accuracy)
    results['stochastic_acc'][key].append(stochastic_accuracy)

def wcb_process_results(results: Dict[str, defaultdict], result_dicts: Dict[str, defaultdict],
                        as_df: bool, use_trace_frequencies: bool, df: pd.DataFrame,
                        test_df: pd.DataFrame) -> Union[Dict[str, defaultdict], pd.DataFrame]:
    if not as_df:
        return results
    
    final_df = wcb_create_final_dataframe(results, test_df)
    
    if use_trace_frequencies:
        wcb_add_trace_frequencies(final_df, df)
    
    return final_df

def wcb_create_final_dataframe(results: Dict[str, defaultdict], test_df: pd.DataFrame) -> pd.DataFrame:
    dataframes = []
    for key, data in results.items():
        # Skip 'alignment' and 'alignment_weights' when creating dataframes
        if key not in ['alignment', 'alignment_weights']:
            df = defaultdict_to_dataframe(data, key)
            dataframes.append(df)
    
    final_df = pd.concat(dataframes, axis=1)
    
    # Add the case:concept:name column
    if 'case:concept:name' not in final_df.columns:
        final_df.insert(0, 'case:concept:name', test_df['case:concept:name'].unique().tolist())
    
    for col in final_df.columns:
        if '_time' in col:
            final_df[col] = final_df[col].astype(float).round(3)
        elif '_nodes_opened' in col:
            final_df[col] = final_df[col].astype(float).round(2)
        elif col == 'case:concept:name':
            continue
        else:
            try:
                final_df[col] = final_df[col].astype(float).round(2)
            except ValueError:
                # If conversion to float fails, keep the column as is
                pass
    
    return final_df

def wcb_add_trace_frequencies(final_df: pd.DataFrame, df: pd.DataFrame) -> Dict[str, float]:
    trace_frequencies = create_case_id_to_frequency_dict(df)
    final_df['frequency'] = final_df['case:concept:name'].apply(lambda x: trace_frequencies.get(x, 0))

    aggregated_results = {}
    columns_to_aggregate = final_df.columns[1:]  # Skip the first column

    for column in columns_to_aggregate:
        if column != 'frequency':
            weighted_sum = (final_df[column] * final_df['frequency']).sum()
            aggregated_results[column] = weighted_sum

    return aggregated_results

In [37]:
res, alignment_dict, (net, init_marking, final_marking), test_df, test_ground_truth, alignment_weights_dict = compare_window_based_baselines(
    filtered_df,
    n_train_traces=5,
    n_test_traces=2,
    train_traces=None,
    test_traces=None,
    window_lengths_lst=[50],
    n_final_markings_lst=None,
    seed_1=97136,
    seed_2=202,
    random_trace_selection=True,
    as_df=True,
    viz_proc_tree=True,
    return_alignment_res_dict=True,
    return_model=True,
    explor_reward=0.000,
    window_overlap=0,
    read_model_from_file=False,
    file_path='',
    activity_mapping_dict=None,
    use_trace_frequencies=False,
    return_agg_res_dict=False,
    reachability_heuristic=False,
    trace_recovery=False,
    map_activities_to_strings=False,
    sftmax_lst=filtered_softmax_lst,
    n_sampling_indices=None,
    non_sync_penalty=1,
    round_precision=3,
    activity_prob_threshold=0.00,
    sequential_sampling=True, 
    return_test_df=True,
    return_test_ground_truth=True,
    cost_function='logarithmic',
    logarithmic_scaling_constant=7,
    allow_train_test_intersection=True,
    return_alignment_weights_dict=True,
    hist_prob_dict=None
)

The train cases are: ['26', '21', '8', '14', '17'] 

The test cases are: ['26', '21'] 

Evaluating variant: n_markings=1, window_len=50


In [41]:
res

,case:concept:name,"marking=1, window_len=50_cost","marking=1, window_len=50_time","marking=1, window_len=50_nodes_opened","marking=1, window_len=50_stochastic_acc","marking=1, window_len=50_argmax_acc"
0,26,15.53,21.887,50690.0,0.70,0.71
1,21,17.33,31.455,77311.0,0.87,0.87
